# 銀イルカの3人のセリフをスクレイピングして，同じところと違うところを

とりま wikiとかから智絵里のセリフをスクレイピングする． 

テキスト解析 -> 出現頻度，


セリフをword2vecで学習して，分散表現を計算できるようにする

分散表現を次元削減して2次元座標上にプロットすることで，単語ごとの類似度を可視化する

銀イルカ3人のセリフをスクレイピング  

doc2vecで学習して，

### requirementis

- python 3.7.1
- pipenv
- mecab
    - ipa
    - neologd
    
usage: `pipenv install`  
python, pipenv, pyenv, mecabのインストール方法は省略

In [1]:
import glob
import re
import time
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 20 

## デレステセリフのスクレイピング

In [3]:
def get_quotes_sute(q):
    """
    デレステで公開されている各アイドルのセリフデータを取得する
    [arg]
    q: int
    [ret]
    df: DataFrame
    """
    html = f"https://starlight.kirara.ca/char/{q}"
    r = requests.get(html)
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text, "lxml")
    texts = soup.find_all("span")
    corpus = []
    for text in texts:
        corpus.append(text.text)
    df = pd.DataFrame(corpus)
    #df[0].value_counts().head(10) # 値の出現頻度をカウント
    df = df[~df.duplicated()] # 重複行をdataframeから除外
    df = df[~df[0].str.match("^[\s|a-zA-Z|<|\+|\d$]")] # 正規表現にマッチするものを除外
    df = df[df[0] != ""]
    return df.reset_index(drop=True)

## モバマスセリフのスクレイピング

In [4]:
def get_quotes_moba(query="緒方智絵里", debug=False):
    """
    モバマスで公開されているセリフ（含 アイドル名、カード名 など）
    をスクレイピングして pandas dataframe にする
    
    [args]
    query: str 省略すると 緒方智絵里 のセリフをスクレイピングする
    debug: Bool Trueにするとスクレイピング数を上限2にする。それだけ もっと良い使い方あるはず...
    [ret]
    df: DataFrame
    """
    
    html = f"https://icws.indigo-bell.com/search?q={query}&st=n"
    r = requests.get(html)
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text, "lxml")
    a = soup.find("a", string="Last　»").get("href")
    if debug == True:
        n = 2
    else:
        n = int(re.search(r"page=(\d+)&", a).group(1)) # クロールするページ数
    
    df = pd.DataFrame()
    for i in range(n): # セリフが登録されてるページ分ループする
        print(f"crawling page: {i + 1}/{n}")
        html = f"https://icws.indigo-bell.com/search?page={i + 1}&q={query}&st=n"
        r = requests.get(html)
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, "lxml")
        table = soup.find_all("tbody",class_="result")[0]
        rows = table.find_all("tr")
        tmp = []
        tm = 5 + random.randint(0, 5)

        for row in rows:
            cell_txt = []
            for cell in row.find_all(["td", "th"]):
                cell_txt.append(cell.text)
            tmp.append(cell_txt)
        df = df.append(tmp)
        #print(f"sleep: {tm}")
        if i != (n - 1):
            time.sleep(tm)
        else:
            continue
    return df.reset_index(drop=True) # indexを振り直してもとのindexも削除

In [5]:
# deresute id
# chieri : 112
# minami : 185
# yui    : 246
# sae    : 132
# hotaru : 161
# yoshino: 262

In [6]:
idol = ["chieri", "112", "緒方智絵里"]

# data/に idol_moba.csv idol_dere.csvが両方なかったらスクレイピングを実行する
# あったらdataframeにする
if not glob.glob(f"data/{idol[0]}_moba.csv") and not glob.glob(f"data/{idol[0]}_dere.csv"):
    print("nai")
    aa = get_quotes_sute(int(idol[1]))
    bb = get_quotes_moba(query=idol[2])
    aa.to_csv(f"data/{idol[0]}_moba.csv")
    bb.to_csv(f"data/{idol[0]}_dere.csv")
else:
    print("aru")
    aa = pd.read_csv(f"data/{idol[0]}_moba.csv", index_col=0)
    bb = pd.read_csv(f"data/{idol[0]}_dere.csv", index_col=0)

aru


In [7]:
aa.head(10)

,0
0,アイドルマスター シンデレラガールズ スターライトステージ!
1,お、おはようございます。アーリーモーニングティー、いかがですか…？
2,お昼だけど…もうちょっとだけ…午後のレッスンの予習をしなくっちゃ…
3,ふぁぁ…こんな時間までお疲れさまです。ホットミルク、飲みますか…？
4,プロデューサーさん…もしお暇なら、私…ステージに、立ちたい…です
5,え…と、あの…ちひろさんが呼んでいましたよ
6,プロデューサーさん宛てに申請だそうです…だ、誰からでしょう…
7,わぁ、イベントが開催されているみたいです。楽しそう…
8,プレゼントが届いているようです…プレゼントって…ステキですよね…
9,あの…おつかれですか？こんなとこで寝ると…風邪ひいちゃいますよ…


In [8]:
bb.head(10)

,0,1,2,3
0,緒方智絵里,緒方智絵里,アイドルコメント,緒方…緒方智絵里……です。あの……その……えぇと…が、がんばります……ので……見捨てないでく...
1,緒方智絵里,緒方智絵里,マイスタジオ,あ……おはようございます
2,緒方智絵里,緒方智絵里,マイスタジオ,あの…あ、なんでもないです…
3,緒方智絵里,緒方智絵里,マイスタジオ,(いい天気…お散歩したいな…)
4,緒方智絵里,緒方智絵里,マイスタジオ,○○プロデューサー、あの……見捨てないでくださいね…?
5,緒方智絵里,緒方智絵里,お仕事,がんばり…ます
6,緒方智絵里,緒方智絵里,お仕事,は、はいっ、お仕事ですね
7,緒方智絵里,緒方智絵里,お仕事,わたし、できるかな…
8,緒方智絵里,緒方智絵里,お仕事,○○プロデューサー……い、いなくならないでくださいね?
9,緒方智絵里,緒方智絵里,お仕事（親愛度UP）,○○プロデューサー、わたし……がんばれました…


## モバ，ステの結合と正規化

In [9]:
# とてもしょーもない(intをわざわざstrとして処理してるのが)
bb["3"] = bb["3"].str.replace("○○", "") # OO (プロデューサー名)部分を落とす
bb["3"] = bb["3"].str.replace("〇〇", "") # OO (プロデューサー名)部分を落とす
# aa + bb のコーパスを作る
docs = pd.Series()
docs = pd.concat([aa["0"], bb["3"]])
docs = docs.str.replace("プロデューサー", "PRODUCER")
docs = docs.str.replace("[Producer]", "PRODUCER", regex=False)
docs = docs.reset_index(drop=True)

In [10]:
docs.to_csv(f'data/{idol[0]}_corpus.csv')

/home/wani/dere/idol_info/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


## 形態素解析

mecabを使う。インストールしていなければめかぶをインストール

neeologdまでインストールする手順はの参考は
- [ubuntu 18.10 に mecab をインストール](https://qiita.com/ekzemplaro/items/c98c7f6698f130b55d53)
- [WSLでMeCabとJupyter Notebookを動かす](https://qiita.com/hironobukawaguchi3/items/61a8664dd26c1da66880)
- [MecabをPythonで使うまで](https://qiita.com/Sak1361/items/47e9ec464ccc770cd65c)

---

$ mecab -d /usr/lib/mecab/dic/mecab-ipadic-neologd


wsl in ubuntu だとうまく動作しない可能性あり．

In [34]:
mecab_path = !echo `mecab-config --dicdir`
def extractNoun(text,
                mecab_mode = rf'-Owakati -d {mecab_path[0]}/mecab-ipadic-neologd',
                noun="名詞"):
    """
    [arg]: str
    [ret]: str
    """
    tagger = MeCab.Tagger(mecab_mode)
    node = tagger.parseToNode(text)
    keywords = []
    while node:
        surface = node.surface # 表層形を表示(str)
        meta = node.feature.split(",") # ['記号', '読点', '*', '*', '*', '*', '、', '、', '、']
        if meta[0] == noun:
            keywords.append(node.surface)
        node = node.next # 次のノードのアドレスを指定
    return " ".join(keywords)

def tokenize(text, wakati=True, dic="neologd"):
    """
    テキストを形態素解析して分かち書きする
    [arg]
    text: str
    [ret]
    term: str
    """
    if dic == "neologd":
        mecab_mode = MeCab.Tagger(rf'-Owakati -d {mecab_path[0]}/mecab-ipadic-neologd') # debugging
    else:
        mecab_mode = MeCab.Tagger(r'-Owakati')
    if wakati == True: # 分かち書きしたものをそのまま取り出す
        term = mecab_mode.parse(text).strip()
    else: # 品詞で
        term = extractNoun(text, mecab_mode)
    return term

In [35]:
import MeCab
text = "テキストを形態素解析して分かち書きする"
print(extractNoun(text))

テキスト 形態素解析 分かち書き


In [36]:
print(f'ipa-dic    ： {tokenize("特急はくたか", dic = "ipa")}')
print(f'neologd-dic： {tokenize("特急はくたか")}')

ipa-dic    ： 特急 は く た か
neologd-dic： 特急 はくたか


## 文章解析

In [37]:
from gensim import corpora, models

In [38]:
#docs_list = [tokenize(row).split() for row in docs]
docs_list = [extractNoun(row).split() for row in docs]
dic = corpora.Dictionary(docs_list)
dic.save_as_text('./data/text.dict')  # 保存

In [ ]:
#gensim.corpora.Dictionary.load_from_text('./data/text.dict')  # 次回からファイルロード可

In [39]:
print(len(dic)) # dictionaryに登録された単語(形態素)数

1552


In [41]:
# 単語出現頻度を数える
# 余談: bashだと $ sort -t$'\t' -k3 -nr text.dict で単語とその出現頻度回数カウントできる
from collections import defaultdict
freq = defaultdict(int)

for text in docs_list:
    for token in text:
        freq[token] += 1
print(sorted(freq.items(), key=lambda x:x[1], reverse=True))

[('さん', 634), ('私', 277), ('ん', 274), ('PRODUCER', 268), ('…。', 163), ('わたし', 127), ('みんな', 122), ('の', 104), ('幸せ', 103), ('こと', 96), ('お仕事', 89), ('アイドル', 76), ('みたい', 73), ('一緒', 65), ('よう', 61), ('智絵里', 54), ('クローバー', 47), ('そう', 46), ('たくさん', 42), ('気持ち', 40), ('気', 39), ('人', 37), ('これ', 36), ('うさぎ', 36), ('今日', 35), ('好き', 35), ('みなさん', 33), ('緊張', 32), ('レッスン', 31), ('それ', 31), ('LIVE', 31), ('今', 30), ('撮影', 30), ('衣装', 29), ('ちゃん', 29), ('大丈夫', 28), ('笑顔', 27), ('お花', 27), ('ステージ', 26), ('心', 26), ('ファン', 26), ('ここ', 25), ('声', 25), ('素敵', 25), ('冬', 23), ('世界', 23), ('京都', 23), ('時間', 22), ('ー', 22), ('前', 22), ('私たち', 22), ('一緒に', 22), ('お願い', 21), ('春', 21), ('ところ', 21), ('綺麗', 21), ('練習', 20), ('空', 20), ('何', 19), ('ため', 19), ('自分', 19), ('子', 19), ('ダメ', 18), ('大切', 18), ('思い出', 18), ('菜々', 18), ('曲', 18), ('がんばり', 16), ('手', 16), ('風', 16), ('日', 16), ('あと', 16), ('たち', 16), ('とき', 15), ('四つ葉のクローバー', 15), ('歌', 15), ('もの', 15), ('勇気', 15), ('事務所', 15), ('夢', 15), ('中',

In [18]:
import matplotlib.pyplot as plt

In [19]:
corpus = [dic.doc2bow(doc) for doc in docs_list] # コーパスの作成
corpora.MmCorpus.serialize('./data/text.mm', corpus)  # 保存

In [20]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

# 折角計算したのでpickleに保存
import pickle
with open('./data/corpus_tfidf.dump', mode='wb') as f:
    pickle.dump(corpus_tfidf, f)

# 次回からはロードできます
# with open('./data/corpus_tfidf.dump', mode='rb') as f:
#     corpus_tfidf = pickle.load(f)

In [22]:
# 文書量にもよりますが、数時間程度かかったりもします。
# '18/12/03追記: LdaMulticore で worker の数を増やせばかなり早くなるかもです
lda = models.LdaModel(corpus=corpus_tfidf, id2word=dic,
                             num_topics=50, minimum_probability=0.001,
                             passes=20, update_every=0, chunksize=10000)
lda.save('./data/lda.model')  # 保存
#lda = gensim.models.LdaModel.load('./data/lda.model')  # 次回からロード可

In [23]:
lda.print_topic(10)

'0.020*"た" + 0.019*"お仕事" + 0.018*"ござい" + 0.018*"ありがとう" + 0.017*"くれ" + 0.016*"て" + 0.015*"京都" + 0.015*"まし" + 0.015*"は" + 0.014*"です"'

In [24]:
for i in range(10):
    print('tpc_{0}: {1}'.format(i, lda.print_topic(i)))

tpc_0: 0.028*"あり" + 0.021*"何" + 0.020*"咲く" + 0.017*"ば" + 0.017*"キラキラ" + 0.012*"さすが" + 0.010*"か" + 0.009*"飼え" + 0.009*"見とれ" + 0.008*"おいしい"
tpc_1: 0.020*"一番" + 0.016*"として" + 0.010*"いけ" + 0.010*"とき" + 0.009*"じゃ" + 0.006*"防犯" + 0.006*"木刀" + 0.006*"用" + 0.006*"芸人" + 0.006*"笑わせる"
tpc_2: 0.082*"へ" + 0.044*"え" + 0.015*"♪" + 0.014*"葉っぱ" + 0.014*"この" + 0.012*"香り" + 0.012*"演技" + 0.012*"の" + 0.012*"届い" + 0.011*"温泉"
tpc_3: 0.018*"音楽" + 0.016*"ひ" + 0.011*"虫" + 0.011*"ゃっ" + 0.011*"髪" + 0.010*"大人" + 0.010*"幸せいっぱい" + 0.009*"言う" + 0.008*"森" + 0.008*"人たち"
tpc_4: 0.016*"飛び込ん" + 0.011*"ファ" + 0.009*"見る" + 0.009*"だら" + 0.009*"呼ん" + 0.008*"世界" + 0.008*"過ぎ" + 0.008*"ち" + 0.008*"ちひろ" + 0.007*"間違え"
tpc_5: 0.031*"下さい" + 0.030*"飾り" + 0.024*"子供っぽい" + 0.019*"よし" + 0.017*"運び" + 0.011*"や" + 0.010*"クローバー" + 0.010*"キレ" + 0.008*"新世界" + 0.008*"この"
tpc_6: 0.023*"ましょ" + 0.021*"見つけ" + 0.019*"楽しみ" + 0.016*"気持ち" + 0.014*"ね" + 0.014*"同じ" + 0.013*"感じ" + 0.013*"♪" + 0.012*"ぱしゃぱしゃ" + 0.012*"外"
tpc_7: 0.035*"綺麗" + 0.029*"優しい" + 0.0